# Impact of Reddit WallStreetBets forum on GME and AMC stocks in early 2021

## Data Source and Import

### Reddit Data Download through API

Initial data import, RRD is Reddit Data for all historical posts and comments in the subreddit group r/wallstreetbets from the beginning to present. 

To download Reddit data, we used the API praw and psaw, the Python Reddit API Wrapper to find the IDs of the historical posts, search for the posts together with comments according to the IDs and download all historical data in the subreddit group r/wallstreetbets. 

Code to download all historical data (Note: this process may take a couple days due to the volume of data) 

The returned data file is a csv file containing around 4 million rows of records, with each record representing a post or its comment. Major variables include reddit post id, created timestamp, title, body, identifier for post or comment. 

The time period covered in this dataset is from the beginning of r/wallstreetbets in 2012 to the end of Aug. 2022. 


### Yahoo Finance 

GME and AMC are daily stock data from December 31, 2019 until October 04, 2022 for those respective symbols, downloaded from Yahoo Finance.

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import warnings
import regex
import altair_transform

warnings.filterwarnings("ignore")
#Read in yahoo finance data
# GME - Date range Aug 31, 2019 - September 30, 2022
GME = pd.read_csv('GME.csv')

# AMC - Date range Aug 31, 2019 - September 30, 2022
AMC = pd.read_csv('AMC.csv')


## Data Cleaning and Manipulation

### Scope Down Time Period and Align Time Zone

Reddit data was provided in PST and Yahoo Finance data was tied to the NYSE in EST. We decided to convert reddit data to EST so that all data is aligned in the same time zone.  

In [2]:
# Read in RedditData 
RRD = pd.read_csv('wsb.csv')

RRD['created']= pd.to_datetime(RRD.created, infer_datetime_format= True)
RRD.sort_values(by = 'created', ascending = True, inplace = True)

In [3]:
RRD['EST'] = RRD['created'] + pd.Timedelta(hours=3)
# RRD.head(10)

The RRD data set is quite large so we needed to scope it down to the relevant time period to free up memory.  

We decided to focus on September, 2019 onward and explore how the short squeeze frenzy started to occur. The reason why we chose September, 2019 as our starting point for investigation is because that's when GameStop’s value started to rise due to Keith Gill – who went by the name of u/DeepF***ingValue – began posting monthly updates about his Long-Term Equity Anticipation Securities (LEAPS) on GameStop via Reddit.

In [4]:
#Scopes Down Reddit Data - Date range 2019-08-31 - 2022-10-01 

RRD['EST'] = pd.to_datetime(RRD['EST'])
mask = (RRD['EST'] > '2019-09-01') & (RRD['EST'] <= '2022-10-01')
RRD = RRD.loc[mask]
df_rrd = RRD

# separate date and time and store them in new cols
df_rrd['date'] = [d.date() for d in df_rrd['EST']]
df_rrd['time'] = [d.time() for d in df_rrd['EST']]
df_rrd.head()

,id,title,url,score,num_comments,body,created,EST,date,time
1874259,eyoxvxv,Comment,cxyfvk,2.0,0.0,Serious answer: neuropeptides,2019-08-31 21:00:24,2019-09-01 00:00:24,2019-09-01,00:00:24
1345358,eyoy0uh,Comment,cy0rp4,14.0,0.0,Go on YouTube and search for Guo Wengui. Whate...,2019-08-31 21:01:29,2019-09-01 00:01:29,2019-09-01,00:01:29
1345356,eyoy8wa,Comment,cy0rp4,73.0,0.0,Counter argument - if China doesn't tamp down ...,2019-08-31 21:03:17,2019-09-01 00:03:17,2019-09-01,00:03:17
1662765,eyoyah3,Comment,cxzo3x,-1.0,0.0,[deleted],2019-08-31 21:03:39,2019-09-01 00:03:39,2019-09-01,00:03:39
3805376,eyoyaz9,Comment,cxjdch,1.0,0.0,4th degree? Category 4. It’s not a burn. Yo...,2019-08-31 21:03:46,2019-09-01 00:03:46,2019-09-01,00:03:46


## Memestock Mentions

Were the top 2 meme stocks in fact GME and AMC as we suspected? Let's look at a list of top 10 popular meme stocks and their respective mention counts in r/WSB

In [5]:
# Using regular expressions to count how many times each individual stock ticker is mentioned in the r/WSB
# subreddit without accidentally over counting. This method is a bit heavier on the processing side, but we 
# get a more accurate count than if we were to use another method that ends up counting / matching incorrect
# strings.

mentions_list = []
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)GME(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)GME(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)AMC(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)AMC(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)BB(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)BB(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)BBBY(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)BBBY(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)NOK(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)NOK(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)EXPR(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)EXPR(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)KOSS(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)KOSS(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)HOOD(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)HOOD(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)BBIG(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)BBIG(?!\S)')).sum())
mentions_list.append((df_rrd.title.str.count(r'(?<!\S)SPCE(?!\S)')).sum() + (df_rrd.body.str.count(r'(?<!\S)SPCE(?!\S)')).sum())
mentions_list

[107110.0,
 43785.0,
 29010.0,
 14679.0,
 9731.0,
 261.0,
 63.0,
 1810.0,
 272.0,
 6086.0]

In [6]:
# https://www.nerdwallet.com/article/investing/meme-stocks
meme_stocks = ["GME","AMC","BB","BBBY","NOK","EXPR","KOSS","HOOD","BBIG","SPCE"]

meme_dict = dict(zip(meme_stocks, mentions_list))
meme_dict
memecounter_df = pd.DataFrame(meme_dict.items(),columns=['memestock','#mentions'])
memecounter_df['% Total Mentions'] = round((memecounter_df['#mentions'] / memecounter_df['#mentions'].sum())*100,2).astype(str)+"%"
# memecounter_df

In [7]:
meme_bars = alt.Chart(memecounter_df).mark_bar().encode(
    x=alt.X('#mentions:Q')
    ,y=alt.Y('memestock:N', sort = '-x'))



meme_text = meme_bars.mark_text(
    align = 'left',
    baseline = 'middle',
    dx = 3).encode(
        text='#mentions:Q'
    )

meme_text2 = meme_bars.mark_text(
    align = 'left',
    baseline = 'middle',
    dx = 3).encode(
        text = '% Total Mentions'
    )

memecount_chart = (meme_bars+meme_text).properties(width=400
,height=300
,title="Memestock # of Mentions in r/WSB from 1/1/2020 to Present")

memepercent_chart = (meme_bars+meme_text2).properties(width=400
,height=300
,title="Memestock % of Mentions in r/WSB from 1/1/2020 to Present")

memecount_chart | memepercent_chart


alt.HConcatChart(...)

Now that we know that GMC and AMC drew the most hype among all meme stocks, let's focus on these two stocks and explore their respective stock mentions comparing with the actual movement of the stocks. Before further analysis begin, we need to manipulate our data to obtain the desired dataframe for visualization.

### Aggregate Daily Data and Count Mentions

Roll up the data into daily data to make it more manageable GME and AMC respectively:

In [8]:
# aggregate data by date, count the number of mention of GME 
gme_body_count = df_rrd.groupby('date')['body'].apply(lambda x: (x.str.contains(r'(?<!\S)GME(?!\S)').sum()))
gme_title_count = df_rrd.groupby('date')['title'].apply(lambda x: (x.str.contains(r'(?<!\S)GME(?!\S)').sum()))
gme_count = pd.merge(gme_body_count, gme_title_count, right_index = True,
               left_index = True)
gme_count = gme_count.reset_index()
gme_count["date"] = pd.to_datetime(gme_count["date"])
gme_count['gme_rrd_count']= gme_count['body'] +gme_count['title']
# gme_count

In [9]:
# aggregate data by date, count the number of mention of AMC 
amc_body_count = df_rrd.groupby('date')['body'].apply(lambda x: (x.str.contains(r'(?<!\S)AMC(?!\S)').sum()))
amc_title_count = df_rrd.groupby('date')['title'].apply(lambda x: (x.str.contains(r'(?<!\S)AMC(?!\S)').sum()))
amc_count = pd.merge(amc_body_count, amc_title_count, right_index = True,
               left_index = True)
amc_count = amc_count.reset_index()
amc_count["date"] = pd.to_datetime(amc_count["date"])
amc_count['amc_rrd_count']= amc_count['body'] + amc_count['title']

# amc_count

In [10]:
overall_gme = gme_count.copy()
overall_amc = amc_count.copy()

overall_gme['symbol'] = 'GME'
overall_amc['symbol'] = 'AMC'

overall_amc = overall_amc[['date','amc_rrd_count','symbol']]
overall_gme = overall_gme[['date','gme_rrd_count','symbol']]

overall_amc = overall_amc.rename(columns={'amc_rrd_count':"count"})
overall_gme = overall_gme.rename(columns={'gme_rrd_count':"count"})

df_final = pd.concat([overall_gme,overall_amc])
# print(df_final)

### Top Two Meme-stock Mentions on WallStreetBets

What happened in terms of reddit mention on GME and AMC since September 2019, when u/DeepF***ingValue began to gain traction on reddit about short squeezing GME? When did the first major spike occur?

In [11]:
# Investigate spikes of meme stock mention on Reddit: Date range 2019-08-31 - 2022-10-01 

fig = alt.Chart(df_final).mark_line().encode(
    x=alt.X('date:T', axis= alt.Axis(title='Date')),
    y= alt.Y('count:Q', axis= alt.Axis(title='Number of Mentions')),
    color='symbol',
    tooltip=['date','count', 'symbol']
).properties(title= 'GME: Reddit Mention',width=600, height=400)

fig

alt.Chart(...)

From the above visualization we can see the obvious spike that occurs in early 2021, so let's focus on 2021-01-01 through 2021-04-01 and see what happened there during the initial spike. 

## How does chatter on the subreddit r/WallStreetBets react with the actual movement of the stock? 

### GME - 3 Month View Reddit Mention with High Price

Filtered GME stock data to the same time period that we identified as first major spike. Created visualization that shows Reddit mention data as bar charts with the corresponding stock high price as line charts. A mouseover interactive feature was added on the graph for easier readability of respective stock price and number of reddit mentions.

In [12]:
# Scope down time frame:GME Reddit
mask = (gme_count['date'] >= '2021-01-01') & (gme_count['date'] <= '2021-04-01')
gme_count_scoped_down = gme_count.loc[mask]

# Narrow down time frame
# Scope down time frame:GME Stock
GME["Date"] = pd.to_datetime(GME["Date"])
mask = (GME['Date'] >= '2021-01-01') & (GME['Date'] <= '2021-04-01')
GME_df = GME.loc[mask]

gme_count_scoped_down = gme_count_scoped_down.rename(columns={"date": "Date"})
df_final_gme = gme_count_scoped_down.join(GME_df.set_index('Date'), on='Date')

# Visualization- 
# GME stock high price from 2021-01-01 - 2021-04-01 and Reddit Count comparison

annotations = [['2021-02-23',3000, 'Jan 28th: Robinhood suspended serivice']]
a_df = pd.DataFrame(annotations, columns=['date','count','note'])

    
annotate_df = pd.DataFrame({
        'x':['2021-01-29','2021-02-05']
        , 'y':[6100, 5500]
    })
annotation_lines = alt.Chart(annotate_df).mark_line(color='black').encode(
            x='x:T'
            , y=alt.Y('y',title='Number of GME Reddit Mention')
            )
    
    
annotate = alt.Chart(a_df).mark_text(fontSize = 12
        ).encode(
            y= alt.value(+70)
            , x = 'date:T'
            , text = 'note'
        )

base = alt.Chart(df_final_gme).encode(
    alt.X('Date:T', axis=alt.Axis(title=None))
    ).properties(
        width=600,
        height=400
    )

gme_reddit = base.mark_bar(color= 'lightblue'
).encode(
    y= alt.Y('gme_rrd_count:Q', axis=None),
    # axis= alt.Axis(title='Number of GME Reddit Mention', titleColor='lightblue')),
    tooltip=['Date', 'gme_rrd_count']
)

GME_high = base.mark_line(
    stroke='green', 
    interpolate='monotone',
    point=alt.OverlayMarkDef(color="green")
).encode(
    y= alt.Y('High:Q', axis= alt.Axis(title='GME Stock High Price ($USD)', titleColor='green')),
    tooltip=['Date', 'High']
)


base_new= alt.layer(gme_reddit, GME_high
    ).properties(width=600,
        height=400,
        title= alt.TitleParams(
            text='GME: Reddit Mention v.s. Stock Movement'
            ,fontSize=20
            , subtitle=['* disconnects in line chart for stock price is a result of non-trading days']
            ,subtitleColor='green'
            # ,subtitleLineHeight=100
            # , subtitlePadding=10
        )
    ).resolve_scale(
        y = 'independent'
    )

alt.hconcat(
    base_new+annotate+annotation_lines
    
).configure_axis(grid=False
    ).configure_axisRight(
        titleColor='lightblue'
        ,labelColor='lightblue'
        # ,labelOpacity=100
        ).configure_axisLeft(
            titleColor='green'
            ,labelColor='green'
        )

alt.HConcatChart(...)

From the graph, we can see that GME stock value had the highest 2429% growth from $4.775 in the beginning of January to its highest price in history of $120.75 on Jan 28, 2021. Overall, the volume in reddit chatter seemed to align with the movement in the stock market with an exception on Jan 28th where stock price ceased to rise as a response to chatter volume. This is due to the fact that trading platform Robinhood temporarily suspended its service on meme stocks such as GME and AMC.

### AMC - 3 Month View Reddit Mention with High Price

Filtered AMC stock data to the same time period we used for GME above:

In [13]:
# Scope down time frame:AMC Reddit
mask = (amc_count['date'] >= '2021-01-01') & (amc_count['date'] <= '2021-04-01')
amc_count_scoped_down = amc_count.loc[mask]

# Narrow down time frame
# # Scope down time frame:AMC Stock
AMC["Date"] = pd.to_datetime(AMC["Date"])
mask = (AMC['Date'] >= '2021-01-01') & (AMC['Date'] <= '2021-04-01')
AMC_df = AMC.loc[mask]

amc_count_scoped_down = amc_count_scoped_down.rename(columns={"date": "Date"})
df_final_amc = amc_count_scoped_down.join(AMC_df.set_index('Date'), on='Date')

# Visualization- 
# AMC stock high price from 2021-01-01 - 2021-04-01 and Reddit Count comparison

annotations = [['2021-02-26',2300, 'Jan 28th: Robinhood suspended serivice']]
a_df = pd.DataFrame(annotations, columns=['date','count','note'])

    
annotate_df = pd.DataFrame({
        'x':['2021-01-29','2021-02-08']
        , 'y':[2500, 2300]
    })
annotation_lines = alt.Chart(annotate_df).mark_line(color='black').encode(
            x='x:T'
            , y=alt.Y('y',title='Number of AMC Reddit Mention')
            )
    
    
annotate = alt.Chart(a_df).mark_text(fontSize = 12
        ).encode(
            y= alt.value(+45)
            , x = 'date:T'
            , text = 'note'
        )

base = alt.Chart(df_final_amc).encode(
    alt.X('Date:T', axis=alt.Axis(title=None))
    ).properties(
        width=600,
        height=400
    )

amc_reddit = base.mark_bar(color= 'lightblue'
).encode(
    y= alt.Y('amc_rrd_count:Q', axis=None),
    # axis= alt.Axis(title='Number of GME Reddit Mention', titleColor='lightblue')),
    tooltip=['Date', 'amc_rrd_count']
)

AMC_high = base.mark_line(
    stroke='green', 
    interpolate='monotone',
    point=alt.OverlayMarkDef(color="green")
).encode(
    y= alt.Y('High:Q', axis= alt.Axis(title='AMC Stock High Price (USD$)', titleColor='green')),
    tooltip=['Date', 'High']
)


base_new= alt.layer(amc_reddit, AMC_high
    ).properties(width=600,
        height=400,
        title= alt.TitleParams(
            text='AMC: Reddit Mention v.s. Stock Movement'
            ,fontSize=20
            , subtitle=['* disconnects in line chart for stock price is a result of non-trading days']
            ,subtitleColor='green'
            # ,subtitleLineHeight=100
            # , subtitlePadding=10
        )
    ).resolve_scale(
        y = 'independent'
    )

alt.hconcat(
    base_new+annotate+annotation_lines
    
).configure_axis(grid=False
    ).configure_axisRight(
        titleColor='lightblue'
        ,labelColor='lightblue'
        # ,labelOpacity=100
        ).configure_axisLeft(
            titleColor='green'
            ,labelColor='green'
        )

alt.HConcatChart(...)

AMC showed a  similar pattern within the same time period. Although comparatively AMC didn’t rise in value as dramatically as GME. It still shows an 825% increase in value from $2.2 in the beginning of January to $20.36 on Jan 28th. Similarly, AMC’s growth was impacted by the event of Robinhood suspending its service.

## Compare Stock Price Change to Reddit Activity

Does high activity on reddit impact solely on a rise to the stock or could it also impact a drop in the stock value? 

In [14]:
# prepare dataframe by calculating the absolute value price change in GME and AMC
# df_final_gme
df_final_gme['Price_Change']= df_final_gme['Close']-df_final_gme['Open']
df_final_gme['Price_Chnage_Abs']= df_final_gme['Price_Change'].abs()
df_final_gme['Price_Chnage_Category']= np.where(df_final_gme['Price_Change']>0, 'Positive', 'Negative')
df_final_gme['Price_Chnage_Category'].mask(df_final_gme['Price_Change'].isna(),np.nan,inplace=True)

df_final_amc
df_final_amc['Price_Change']= df_final_amc['Close']-df_final_amc['Open']
df_final_amc['Price_Chnage_Abs']= df_final_amc['Price_Change'].abs()
df_final_amc['Price_Chnage_Category']= np.where(df_final_amc['Price_Change']>0, 'Positive', 'Negative')
df_final_amc['Price_Chnage_Category'].mask(df_final_amc['Price_Change'].isna(),np.nan,inplace=True)

# df_final_amc

The following graph shows the dollar amount price change of the stock represented by a bar chart. The color of the bar reflects whether or not the change was in a positive direction or negative. Layered into the visualization is a line chart showing the daily reddit mentions.

In [15]:
# GME vis
selection = alt.selection_multi(fields=['Price_Chnage_Category'], bind='legend')

base = alt.Chart(df_final_gme).encode(
    alt.X('Date:T', axis=alt.Axis(title=None))
    ).properties(
        width=600,
        height=400
    )

gme_price_change = base.mark_bar(opacity= 0.7
).encode(
    y= alt.Y('Price_Chnage_Abs:Q', axis= alt.Axis(title='GME Stock Price Change ($USD)', titleColor='green', titleOpacity=0.7))
    ,color= alt.Color('Price_Chnage_Category:N', scale= alt.Scale(domain = ['Negative','Positive'], range = ['red','green']))
    ,opacity=alt.condition(selection, alt.value(0.3), alt.value(0.1))
    ,tooltip=['Date', 'Price_Chnage_Category', 'gme_rrd_count']
).add_selection(selection)

GME_rrd = base.mark_line(
    stroke='blue', 
    interpolate='monotone',
    point=alt.OverlayMarkDef(color="blue")
).encode(
    y= alt.Y('gme_rrd_count:Q', axis= alt.Axis(title='Number of GME Reddit Mentions', titleColor='blue'))
    ,tooltip=['Date', 'Price_Chnage_Category', 'gme_rrd_count']
)

alt.layer(gme_price_change, GME_rrd).properties(title= alt.TitleParams(
    text= 'GME: Price Change v.s. Reddit Activity'
    , fontSize=20)
    ).resolve_scale(
    y = 'independent'
).configure_axis(
    ).configure_axisRight(
        titleColor='blue'
        ,labelColor='blue'
        # ,labelOpacity=100
        ).configure_axisLeft(
            titleColor='green'
            ,labelColor='green'
        )

alt.LayerChart(...)

We can see that for most major spikes in positive stock movement we see a corresponding spike in reddit chatter on wallstreetbets, when the stock price goes down, we see that interest is lost and reddit chatter ceases. However, within this chart we can also see something that would suggest otherwise - the large price change bars around Mar 25th occurred both in a negative than positive direction and no additional chatter happened at that time period. 

This data suggests that there is a relationship between the jump in stock price and the wallstreetbets chatter, but not one strong enough to suggest that stock price was heavily influenced by chatter. For example, the large spike on Jan 26th was heavily influenced by the chatter leading up to it, but after that point the cat was out of the bag and everyone heard about short squeezing GME. We would go even further to suggest that after that spike that positive stock price changes seems to be the one influencing wallstreetbets chatter.

## Compare AMC Price Change to Reddit Activity

The following graph shows the dollar amount price change of the stock represented by a bar chart. The color of the bar reflects whether or not the change was in a positive direction or negative. Layered into the visualization is a line chart showing the daily reddit mentions.

In [16]:
# AMC vis
selection = alt.selection_multi(fields=['Price_Chnage_Category'], bind='legend')

base = alt.Chart(df_final_amc).encode(
    alt.X('Date:T', axis=alt.Axis(title=None))
    ).properties(
        width=600,
        height=400
    )

amc_price_change = base.mark_bar(opacity= 0.7
).encode(
    y= alt.Y('Price_Chnage_Abs:Q', axis= alt.Axis(title='AMC Stock Price Change ($USD)', titleColor='green', titleOpacity=0.7))
    ,color= alt.Color('Price_Chnage_Category:N', scale= alt.Scale(domain = ['Negative','Positive'], range = ['red','green']))
    ,opacity=alt.condition(selection, alt.value(0.3), alt.value(0.1))
    ,tooltip=['Date', 'Price_Chnage_Category', 'amc_rrd_count']
).add_selection(selection)

AMC_rrd = base.mark_line(
    stroke='blue', 
    interpolate='monotone',
    point=alt.OverlayMarkDef(color="blue")
).encode(
    y= alt.Y('amc_rrd_count:Q', axis= alt.Axis(title='Number of AMC Reddit Mentions', titleColor='blue'))
    ,tooltip=['Date', 'Price_Chnage_Category', 'amc_rrd_count']
)

alt.layer(amc_price_change, AMC_rrd).properties(title= alt.TitleParams(
    text= 'AMC: Price Change v.s. Reddit Activity'
    , fontSize=20)
    ).resolve_scale(
    y = 'independent'
).configure_axis(
    ).configure_axisRight(
        titleColor='blue'
        ,labelColor='blue'
        # ,labelOpacity=100
        ).configure_axisLeft(
            titleColor='green'
            ,labelColor='green'
        )

alt.LayerChart(...)

In the AMC chart we can see similar patterns regarding positive stock movement and reddit mentions, as well as the outlier in March. Another insight we could draw is that reddit chatter volume seemed to show some after the fact effect where discussion on reddit seemed to be more active after there was high activity in the stock market. For example, the three green bars leading up to Feb 25th. This brought us to wonder when exactly were reddit chatter most active- was it during the trading hour while there was high movement in the market or after the market was closed?

## Count of Reddit Hourly Activity 

We would like to explore at what time of the day is Reddit the most active. So we calculated the average count number of posts and their comments per hour for all reddit r/wallstreetbets posts and for each of the top 2 mostly mentioned stocks, for the period of 01/01/2021 to 04/01/2021.

Data Manipulation Steps: 

1. Filtered the reddit wsb dataset by time period 01/01/2021 - 04/01/2021.

undefined. Dropped NaNs for any posts that don’t have a valid url, since those have been deleted either by the user or the system.  

undefined. Grouped by hour and counted the average total of posts and comments per hour for all reddit posts, as well as for the top 2 stock tickers GME and AMC. 

In [17]:
timerange = ['2021-01-01','2021-04-01']
mask_act = (RRD['EST'] >= timerange[0]) & (RRD['EST'] <= timerange[1])
RRD_act = RRD.loc[mask_act].dropna(subset = ['url']) #drop posts that have been deleted - url is nan 
RRD_act['mentions'] = RRD_act['title'] + " " + RRD_act['body']
times = pd.to_datetime(RRD['EST'])
daydiff = (pd.to_datetime(timerange[1]) - pd.to_datetime(timerange[0])).days
all_df = round(RRD_act.groupby([times.dt.hour]).count()/daydiff,0)
all_df['hour'] = all_df.index
all_df = all_df[['hour','mentions']]
all_df['Stock'] = 'All'
all_df.head()

GME_act = round(RRD_act.groupby([times.dt.hour])['mentions'].apply(lambda x: x.str.count(r'(?<!\S)GME(?!\S)').sum())/daydiff,0)
GME_act = GME_act.reset_index().rename(columns = {'EST':'hour'})
GME_act['Stock'] = 'GME'

AMC_act = round(RRD_act.groupby([times.dt.hour])['mentions'].apply(lambda x: x.str.count(r'(?<!\S)AMC(?!\S)').sum())/daydiff,0)
AMC_act = AMC_act.reset_index().rename(columns = {'EST':'hour'})
AMC_act['Stock'] = 'AMC'



all_df = pd.concat([all_df,GME_act])
all_df = pd.concat([all_df,AMC_act])


In [18]:
#create visualization on a stackbar 
palette = alt.Scale(domain=['AMC','GME','All'], range=["green", "lightgreen",'lightblue'])

#adding interactivity 
selector = alt.selection_single(encodings=['x', 'color'])


stackbar = alt.Chart(all_df).mark_bar(size = 25,align='left').encode(
    x=alt.X('hour',scale=alt.Scale(domain=[2, 22]),title = "Time of Day (24-Hour Clock EST)"),
    y= alt.Y('mentions:Q', axis= alt.Axis(title='Number of Reddit Posts')),
    color=alt.condition(selector, 'Stock', alt.value('lightgray'),scale=palette),
    tooltip = ['Stock','mentions']
 ).properties(width = 700, height = 330,title='Count of Reddit Hourly Activities').add_selection(
     selector
 ).configure_axis(grid=False) #click to select a bar. Double click to select all 

stackbar

alt.Chart(...)

In the above visualization, we use a stacked bar highlighting activities for GME and AMC in light green and dark green respectively. We can see that during the evenings, Reddit is much more active than during the daytime. Since Reddit is a casual discussion forum that people are more likely to use after work, this pattern is aligned with most people’s social media usage behavior. 

We can also infer from the bar chart that the discussions on the stock GME and AMC are consistent with the overall pattern. 


## Correlation Analysis and Visualization

### GME  - Reddit Mention and Stock Price Correlation Analysis

We conduct a correlation analysis to see if there is a relationship between reddit posts and the actual movement of the stock and how strong that relationship may be for GME. We estimate the Pearson correlation coefficient to quantify the direction and strength of the linear association between the Yahoo finance and Reddit wsb datasets. 

Data Manipulation Steps: 

1. Filtered both the Yahoo finance and Reddit wsb datasets by time period 01/01/2021 - 04/01/2021. 

undefined. Merged the two dataframes on the stock ticker for the stock price and the count of mentions in both the body and title, including text that contains the ticker in either uppercase and lowercase letter.  

In [19]:
#correlation - GME 

timerange = ['2021-01-01','2021-04-01'] #timezone is based on EST  

# Reddit Data Date range 01/01/2021 - 04/01/2021
maskcor = (RRD['EST'] >= timerange[0]) & (RRD['EST'] <= timerange[1])
RRD_cor = RRD.loc[maskcor]
RRD_cor['Date'] = [d.date() for d in RRD_cor['EST']]


# aggregate data by date, count the number of mention of GME 
RRD_cor.groupby(pd.Grouper(key='Date', axis=0, sort=True)).count()
RRD_cor['mentions'] = RRD_cor['title'] + " " + RRD_cor['body']

GME_count = RRD_cor.groupby('Date')['mentions'].apply(lambda x: x.str.count(r'(?<!\S)GME(?!\S)').sum())
GME_count = GME_count.reset_index()


# Yahoo GME - Date range 01/01/2021 - 04/01/2021
GME["Date"] = pd.to_datetime(GME["Date"])
maskgme = (GME['Date'] >= timerange[0]) & (GME['Date'] <= timerange[1])
GME_stock = GME.loc[maskgme][['Date','High']]
GME_stock['Date'] = [d.date() for d in GME_stock['Date']]




merge_df_GME = GME_stock.merge(GME_count, on = 'Date')
merge_df_GME = merge_df_GME.rename(columns = {'High':'StockPrice','mentions':'Mention_Count'})
print(merge_df_GME.corr(),2)
GME_cor = round(merge_df_GME.corr(),2)['Mention_Count'][0]
print(f'The correlation coefficient of stock price and count of reddit mentions for GME is {GME_cor}, indicating a positive correlation')




               StockPrice  Mention_Count
StockPrice       1.000000       0.610431
Mention_Count    0.610431       1.000000 2
The correlation coefficient of stock price and count of reddit mentions for GME is 0.61, indicating a positive correlation


In [20]:
#vis of GME correlation 
merge_df_GME["Date"] = pd.to_datetime(merge_df_GME["Date"])
merge_df_GME['DayofWeek'] = merge_df_GME["Date"].dt.day_name()

gme_cor_base = alt.Chart(merge_df_GME).mark_circle(size=80).encode(
    x=alt.X('StockPrice:Q'),
    y= alt.Y('Mention_Count:Q', axis= alt.Axis(title='Number of GME Reddit Mention')),
    color = alt.Color('DayofWeek',title = 'Day of Week',sort = ['Monday','Tuesday','Wednesday','Thursday','Friday']),
    tooltip=['Date', 'Mention_Count','StockPrice']
)

gme_cor_reg = gme_cor_base.transform_regression('StockPrice','Mention_Count').mark_line().encode(color=alt.value("green"))
params = altair_transform.extract_data(gme_cor_reg)
x0 = params.iloc[0]['StockPrice']
x1 = params.iloc[1]['StockPrice']

y0 = params.iloc[0]['Mention_Count']
y1 = params.iloc[1]['Mention_Count']


a = (y1 - y0)/(x1 - x0)
b = y1 - a*x1 


midprice = merge_df_GME.StockPrice.median() + 30
midmentions = merge_df_GME.Mention_Count.median() + 800

text = alt.Chart({'values':[{'StockPrice': midprice, 'Mention_Count': midmentions}]}).mark_text(
    text='y = ' + '{:.2f}'.format(b) + ' + {:.2f}'.format(a) + 'x', angle=342, color = 'red'
).encode(
    x='StockPrice:Q', y='Mention_Count:Q'
)


gme_cor_chart = gme_cor_base + gme_cor_reg + text
gme_cor_chart.properties(width = 550, height = 330,
    title='Scatter Plot of Number of GME Reddit Mentions and the Stock Price').configure_axis(grid=False)



alt.LayerChart(...)

The correlation coefficient of stock price and count of reddit mentions for GME is 0.61, indicating a positive correlation between the two variables. 

In addition, we wanted to explore if there is any pattern embedded in the day of week. So we add color coding for each day of week in the above scatter plot. The color pattern is pretty random according to the above plot so we don’t have any conclusion for this. 


### AMC - Reddit Mention and Stock Price Correlation Analysis

In [21]:
#correlation - AMC 


# aggregate data by date, count the number of mention of AMC 

AMC_count = RRD_cor.groupby('Date')['mentions'].apply(lambda x: x.str.count(r'(?<!\S)AMC(?!\S)').sum())
AMC_count = AMC_count.reset_index()


# Yahoo AMC - Date range 01/01/2021 - 04/01/2021
AMC["Date"] = pd.to_datetime(AMC["Date"])
maskAMC = (AMC['Date'] >= timerange[0]) & (AMC['Date'] <= timerange[1])
AMC_stock = AMC.loc[maskAMC][['Date','High']]
AMC_stock['Date'] = [d.date() for d in AMC_stock['Date']]




merge_df_AMC = AMC_stock.merge(AMC_count, on = 'Date')
merge_df_AMC = merge_df_AMC.rename(columns = {'High':'StockPrice','mentions':'Mention_Count'})
print(merge_df_AMC.corr(),2)
AMC_cor = round(merge_df_AMC.corr(),2)['Mention_Count'][0]
print(f'The correlation coefficient of stock price and count of reddit mentions for AMC is {AMC_cor}, indicating a positive correlation')


               StockPrice  Mention_Count
StockPrice       1.000000       0.608222
Mention_Count    0.608222       1.000000 2
The correlation coefficient of stock price and count of reddit mentions for AMC is 0.61, indicating a positive correlation


In [22]:
#vis of AMC correlation 
merge_df_AMC["Date"] = pd.to_datetime(merge_df_AMC["Date"])
merge_df_AMC['DayofWeek'] = merge_df_AMC["Date"].dt.day_name()

AMC_cor_base = alt.Chart(merge_df_AMC).mark_circle(size=80).encode(
    x=alt.X('StockPrice:Q'),
    y= alt.Y('Mention_Count:Q', axis= alt.Axis(title='Number of AMC Reddit Mention')),
    color = alt.Color('DayofWeek',title = 'Day of Week',sort = ['Monday','Tuesday','Wednesday','Thursday','Friday']),
    tooltip=['Date', 'Mention_Count','StockPrice']
)

AMC_cor_reg = AMC_cor_base.transform_regression('StockPrice','Mention_Count').mark_line().encode(color=alt.value("green"))
params = altair_transform.extract_data(AMC_cor_reg)
x0 = params.iloc[0]['StockPrice']
x1 = params.iloc[1]['StockPrice']

y0 = params.iloc[0]['Mention_Count']
y1 = params.iloc[1]['Mention_Count']


a = (y1 - y0)/(x1 - x0)
b = y1 - a*x1 


midprice = merge_df_AMC.StockPrice.median() + 5
midmentions = merge_df_AMC.Mention_Count.mean() + 500

text = alt.Chart({'values':[{'StockPrice': midprice, 'Mention_Count': midmentions}]}).mark_text(
    text='y = ' + '{:.2f}'.format(b) + ' + {:.2f}'.format(a) + 'x', angle=340,color = 'red'
).encode(
    x='StockPrice:Q', y='Mention_Count:Q'
)


AMC_cor_chart = AMC_cor_base + AMC_cor_reg + text
AMC_cor_chart.properties(width = 550, height = 330,
    title='Scatter Plot of Number of AMC Reddit Mentions and the Stock Price').configure_axis(grid=False)


alt.LayerChart(...)

We also do the same for AMC. 

The correlation coefficient of stock price and count of reddit mentions for AMC is 0.61, indicating a positive correlation between the two variables. 

Again, we are trying to find the pattern for the day of week, but we don’t see any clear relationship between the day of week and mentions of the stock. 

## Summary of Challenges and Future Opportunities 

In this project, we have conducted various analyses to explore and visualize the relationship between stock price change and the activities on reddit r/wallstreetbets forum. We concluded that there is positive correlation between the two datasets, for the top 2 meme stocks GME and AMC, during the period of 01/01/2021 - 04/01/2021, when meme stocks took off. 

During the project we had encountered a couple of challenges. The first one we faced was that the Kaggle data we originally chose for reddit wsb mentions only  has data through Aug. 2021. So we were missing a big chunk of data for the most recent year. We ended up figuring out how to use reddit api praw and psaw to download the full dataset for the whole subgroup r/wallstreetbets so we are able to conduct the analysis for the period we chose. 

Another challenge was that we wanted to see the correlation between yahoo finance and reddit mentions per hour. However, Yahoo finance doesn’t provide us with hourly historical stock prices. We ended up calculating the difference between the open and close price to support our analysis in the above section: Compare GME/AMC Price Change to Reddit Activity.

We also wanted to explore the impact on the stock price of an important person: Keith Gill, who started the GameStop madness back in September 2019. According to his wikipedia, his posts under the username “DeepFuckingValue” on Reddit were cited as a driving factor in the GameStop short squeeze of Jan. 2021 and the subsequent trading craziness in meme stocks. However our dataset has all user info anonymized so we weren’t able to conduct the exploration on his behaviors in an efficient way. This also creates another future research opportunity for us after we figure out how to obtain the historical data for a specific user on Reddit. 

In the future, we would like to apply more advanced techniques using machine learning methods to conduct predictions for meme stock spike from the reddit wsb posting activities. And we are also thinking about looking at stock trade volumes and correlating that to mentions of reddit. 


## Reference

Reddit API: https://github.com/gabrielpreda/reddit_extract_content

Wikipedia: https://en.wikipedia.org/wiki/Keith_Gill

News sources:

- Robinhood suspended service on Jan 28, 2021: https://www.cnbc.com/2021/01/28/robinhood-interactive-brokers-restrict-trading-in-gamestop-s.html

- Top 10 popular memestocks: https://www.nerdwallet.com/article/investing/meme-stocks

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e06d726f-a243-4f0d-8026-495c633ea3dd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>